In [ ]:
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import *
init_notebook_mode()

# Boundary Value Problem in 1D

## The Strong Form

Here I solve the following problem:

$$
\frac{d^2u}{dx^2} + \sin\left(\pi x\right) = 0
$$

$$ u \in \left[0,1\right] $$

$$ u(0) = u(1) = 0 $$

This is probably as simple as it gets! We can solve this analytically and get the solution

$$u(x) = \frac{\sin\left(\pi x\right)}{\pi^2}$$

## The Weak Form

The above equations are the strong form of the problem, in order to get a finite element scheme, we need to find the weak form. For this we multiply the strong form with an arbitrary test function $v(x)$ and integrate over the whole domain $\Omega = \left[0,1\right]$:

$$\int_\Omega \frac{d^2u(x)}{dx^2} v(x) dx = - \int_\Omega f(x) v(x) dx$$

with $f(x) = \sin\left(\pi x\right)$.

Now we can use integration by parts to move one of the derivatives from $u(x)$ to $v(x)$ to arrive at the weak form:

$$\left[\frac{du(x)}{dx}v(x)\right]_0^1 -  \int_\Omega \frac{du(x)}{dx} \frac{dv(x)}{dx} dx = - \int_\Omega f(x) v(x) dx$$

The first term is the boundary term comming out of the integration by parts. As we do have Dirichilet boundary conditions we require all our test functions $v(x)$ to satisfy the boundary conditions $v(0) = v(1) = 0$. This makes the first term zero and we get (using $u^\prime = \frac{du(x)}{dx}$):

$$\int_\Omega u^\prime v^\prime dx = \int_\Omega fvdx$$

## The Basis Functions

We will approximate (project) the true solution $u(x)$, which is an infintely dimensional function, onto a finite dimensional function space $\Phi$. We construct this function space from piecewise linear basis functions $\phi_i(x)$. 

Let's first split the domain of the problem into $n$ elements:

In [ ]:
n = 3
xnodes = np.linspace(0, 1, n+1)

Now we construct for each of these elements a basis function

$$\phi_i(x) = \frac{x -x_{i-1}}{x_{i}-x_{i-1}}\quad\quad x_{i-1} \lt x \le x_{i}$$ 

$$\phi_i(x) = \frac{x_{i+1} -x}{x_{i+1}-x_{i}}\quad\quad x_{i} \lt x \le x_{i+1}$$

for all other $x$ the function is 0.

This is a funciton that creates hat functions and we use it to create our basise



In [ ]:
def createHatFunction(i, n, xnodes):
    phi = (lambda x : (x- xnodes[i-1])/(xnodes[i]-xnodes[i-1]) if (x > xnodes[i-1]) and (x <= xnodes[i])else ( xnodes[i+1] - x)/(xnodes[i+1]-xnodes[i])if (x>=xnodes[i]) and (x < xnodes[i+1])
           else 0)
    return phi

basis = []
for i in range(0,n+1):
    basis.append(createHatFunction(i, n, xnodes))


If we have $n$ elements we need a $n+1$ dimensional basis.

Let's plot the basis functions over the domain:

In [ ]:
data = []
domain = np.linspace(0, 1, 100)
i = 0
for phi in basis:
    data.append(Scatter(x=domain, y =[phi(x) for x in domain], name="phi %d"%i))
    i = i + 1
iplot(data)

In order to satisfy the boundary conditions we have to remove $\phi_0$ and $\phi_3$ from our basis.

In [ ]:
basis = [createHatFunction(1, n, xnodes), createHatFunction(2, n, xnodes)]

## Building the Stiffness Matrix

So with our basis functions we approximate the $u(x) \approx U(x) = a_1\phi_1(x) + a_2\phi_2(x)$. When we insert this into our weak formulation we get:

$$\int_\Omega \left( a_1\phi^\prime_1(x) + a_2\phi^\prime_2(x)\right)v^\prime(x)dx = \int_\Omega f v(x) dx$$

Which means we have one equation and two unknowns! In order to get the two equations that we need to get a determined system, we use the two basis functions for $v(x)$ so that we get:

$$\int_\Omega \left( a_1\phi^\prime_1(x) + a_2\phi^\prime_2(x)\right)\phi_1^\prime(x)dx = \int_\Omega f \phi_1^\prime(x) dx$$
$$\int_\Omega \left( a_1\phi^\prime_1(x) + a_2\phi^\prime_2(x)\right)\phi_2^\prime(x)dx = \int_\Omega f \phi_2^\prime(x) dx$$


## Basis Functions: The Element View

Previously we have viewed the basis functions on the full domain of the problem. The variable $x$ we used there is called the _global_ variable. But it is advantageous to move to an element centric view. For this we split the domain into elements and using a variable transformation we have each element sit between -1 and 1. These are called the _natural_ variables.

So assuming we have the nodes in global variables $x_1, x_2, \dots x_n$, we define the natural variable for the element from $x_i$ to $x_{j}$ ($x_j > x_i$):

$$\xi(x) = \frac{2x - x_i - x_{j}}{x_{j} - x_i}$$

Note that $x_i$ does not need to be adjacent to $x_j$: how many nodes we have in an element depends on the order of the interpolation polynomial! For linear hat functions we have indeed $x_j = x_{i+1}$, so we just have two nodes to an element.

Note that we have

$$\xi(x_i) = -1$$
$$\xi(x_j) = +1$$

Now we can formulate our hat functions in natural variables:

$$\phi_1(\xi) = \frac{1}{2}\left(1-\xi\right)$$
$$\phi_2(\xi) = \frac{1}{2}\left(1+\xi\right)$$

Note that $\phi_1(-1) = 1$ and $\phi_2(+1) = 1$ while the basis functions on the other node are zero.

_This is common to all basis functions: We have as many basis functions as we have nodes in each element and each basis function is 1 at just one node, 0 at all others_

In [ ]:
# cubic functions in natural coordinates
phi1 = (lambda xi : -0.5*xi*(1.0-xi))
phi2 = (lambda xi : +0.5*xi*(1.0+xi))
phi3 = (lambda xi : (1.0-xi)*(1.0+xi))


In [ ]:
domain = np.linspace(-1, 1, 100)
data = []
data.append(Scatter(x=domain, y=[phi1(x) for x in domain], name="Phi 1"))
data.append(Scatter(x=domain, y=[phi2(x) for x in domain], name="Phi 2"))
data.append(Scatter(x=domain, y=[phi3(x) for x in domain], name="Phi 3"))
iplot(data)